# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output_data/city_data.csv")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Arlit,18.7369,7.3853,80.51,18,2,15.70,NE,1666815146
1,Butaritari,3.0707,172.7902,81.99,75,100,13.85,KI,1666815146
2,Taltal,-25.4000,-70.4833,64.00,64,23,6.60,CL,1666815147
3,Marsh Harbour,26.5412,-77.0636,78.44,63,7,10.51,BS,1666815147
4,Atuona,-9.8000,-139.0333,77.22,74,23,19.44,PF,1666815148
...,...,...,...,...,...,...,...,...,...
546,Rio Branco do Sul,-25.1900,-49.3142,81.25,42,0,1.61,BR,1666816059
547,Luba,3.4568,8.5547,80.49,85,100,5.06,GQ,1666816059
548,Mount Isa,-20.7333,139.5000,71.37,88,85,6.91,AU,1666816060
549,Pinawa,50.1489,-95.8811,38.79,53,98,4.85,CA,1666816060


In [3]:
city_df["Lat"].count()

551

In [4]:
city_df["Lng"].count()

551

In [5]:
city_df["Humidity"].count()

551

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#For API Key in gmaps
gmaps.configure(api_key=g_key) 

In [7]:
locations = city_df[['Lat', 'Lng']]
humidity=  city_df['Humidity']

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating = False)
heatmap_layer.max_intensity = 70
heatmap_layer.point_radius = 3

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:

ideal_weather_df =city_df.loc[(city_df["Wind Speed"]<=10) & (city_df["Cloudiness"]==0) & (city_df["Max Temp"]<=80) & (city_df["Max Temp"]>=70)].dropna()


ideal_weather_df 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,Nizwá,22.9333,57.5333,76.86,30,0,3.76,OM,1666815158
35,Roebourne,-20.7833,117.1333,72.70,46,0,4.97,AU,1666815159
48,Salalah,17.0151,54.0924,77.09,57,0,4.61,OM,1666815163
118,Castro,-24.7911,-50.0119,73.33,51,0,5.28,BR,1666815314
174,Bethel,41.3712,-73.4140,74.25,72,0,6.91,US,1666815399
181,Ubari,26.5921,12.7805,70.77,22,0,7.94,LY,1666815402
199,Puerto del Rosario,28.5004,-13.8627,74.95,78,0,8.05,ES,1666815468
261,Ajdabiya,30.7554,20.2263,70.18,70,0,9.62,LY,1666815555
269,Nageshwari,25.9377,89.7148,70.57,77,0,1.97,BD,1666815559
276,Parker,32.7668,-97.8003,73.85,35,0,5.01,US,1666815622


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = None
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,Nizwá,22.9333,57.5333,76.86,30,0,3.76,OM,1666815158,None
35,Roebourne,-20.7833,117.1333,72.70,46,0,4.97,AU,1666815159,None
48,Salalah,17.0151,54.0924,77.09,57,0,4.61,OM,1666815163,None
118,Castro,-24.7911,-50.0119,73.33,51,0,5.28,BR,1666815314,None
174,Bethel,41.3712,-73.4140,74.25,72,0,6.91,US,1666815399,None
181,Ubari,26.5921,12.7805,70.77,22,0,7.94,LY,1666815402,None
199,Puerto del Rosario,28.5004,-13.8627,74.95,78,0,8.05,ES,1666815468,None
261,Ajdabiya,30.7554,20.2263,70.18,70,0,9.62,LY,1666815555,None
269,Nageshwari,25.9377,89.7148,70.57,77,0,1.97,BD,1666815559,None
276,Parker,32.7668,-97.8003,73.85,35,0,5.01,US,1666815622,None


In [10]:

# link : https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters

params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #getting the lat and lng value from df
    lat=row['Lat']
    lng=row['Lng']
    params['location'] = f"{lat},{lng}"
    #base url for the request
    base ='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    #making requests to get response
    hotel_info = requests.get(base, params = params).json()
    #try and except
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result... skipping.")
        
hotel_df

Missing result... skipping.
Missing result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,Nizwá,22.9333,57.5333,76.86,30,0,3.76,OM,1666815158,Alila Jabal Akhdar
35,Roebourne,-20.7833,117.1333,72.70,46,0,4.97,AU,1666815159,Dampier Mermaid Hotel Karratha
48,Salalah,17.0151,54.0924,77.09,57,0,4.61,OM,1666815163,Crowne Plaza Resort Salalah
118,Castro,-24.7911,-50.0119,73.33,51,0,5.28,BR,1666815314,Bourbon Ponta Grossa Convention Hotel
174,Bethel,41.3712,-73.4140,74.25,72,0,6.91,US,1666815399,Hampton Inn & Suites Yonkers
181,Ubari,26.5921,12.7805,70.77,22,0,7.94,LY,1666815402,None
199,Puerto del Rosario,28.5004,-13.8627,74.95,78,0,8.05,ES,1666815468,Hotel Riu Palace Tres Islas
261,Ajdabiya,30.7554,20.2263,70.18,70,0,9.62,LY,1666815555,الجلاء للشقق الفندقية
269,Nageshwari,25.9377,89.7148,70.57,77,0,1.97,BD,1666815559,Grand Palace Hotel
276,Parker,32.7668,-97.8003,73.85,35,0,5.01,US,1666815622,Hilton Garden Inn Fort Worth Alliance Airport


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))